### I Pad Air 5

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# pip install deep-translator

In [3]:
import re
import nltk
import spacy
import codecs
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from textblob import TextBlob
import plotly.graph_objects as go
from nltk.corpus import stopwords
from mysutils.text import remove_urls
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from deep_translator import GoogleTranslator
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,BaggingClassifier
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold,GridSearchCV
# nltk.download('all')

2023-02-04 16:40:12.067235: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-04 16:40:12.223121: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-04 16:40:12.726104: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-04 16:40:12.726168: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [4]:
df=pd.DataFrame(columns=['name','date','rating','subject','review'],)

In [5]:
df

,name,date,rating,subject,review


In [6]:
# web scrapping

for i in range(1,1000):    # dont know how many pages contains reviews 
                           # so using iteration
    url=f"https://www.amazon.in/Apple-iPad-Air-10-9-inch-27-69-Wi-Fi/product-reviews/B09V471196/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={i}"

    lists=[]
    
    # below snippet will try untill we successfully get the website
    while not lists:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        lists = soup.find_all("div", class_="a-section celwidget")
    
    
    # Now scrapping the reviews
    for review in lists:
        name = review.find('span',class_='a-profile-name').text
        date = review.find('span',class_='a-size-base a-color-secondary review-date').text
        rating =review.find('span',class_='a-icon-alt').text
        subject = review.find('span',class_=None).text
        review = review.find('span',class_='a-size-base review-text review-text-content').text
        
        df.loc[len(df)] = [name, date, rating, subject, review]
    
    # This will break the loop if 'Next page' button is greyed out
    if len(soup.find_all('li',{"class":'a-disabled a-last'})):
        print('last page no is :',i)
        break
        

last page no is : 15


In [7]:
df.head()

,name,date,rating,subject,review
0,Masum Ahmed,Reviewed in India 🇮🇳 on 31 January 2023,5.0 out of 5 stars,Awesome,
1,ImJ,Reviewed in India 🇮🇳 on 8 December 2022,4.0 out of 5 stars,Its an Apple iPad,\nLoved the decent battery life and speakers a...
2,Mrinal,Reviewed in India 🇮🇳 on 12 January 2023,5.0 out of 5 stars,Nothing comes close to this.,\nThere is literally no competition to iPad Ai...
3,Biresh Mohapatra,Reviewed in India 🇮🇳 on 21 January 2023,5.0 out of 5 stars,Smooth running. Low screen quality,\nI don't like its screen quality and 🔋 life. ...
4,parmesh rawte,Reviewed in India 🇮🇳 on 30 January 2023,5.0 out of 5 stars,best,\nbest ipad. for students\n


In [8]:
# Now making new_df for sentiment analysis
new_df=df.copy()

In [9]:
# In date column, we dont need 'reviewed in India' text
# so removing them

In [10]:
kkkk='Reviewed in India 🇮🇳 on 21 January 2023'

In [11]:
kkkk.find('on')

21

In [12]:
kkkk[21+3]

'2'

In [13]:
# So now removing all texts from these points
new_df['date']=new_df['date'].apply(lambda x: x[x.find('on')+3:])

In [14]:
new_df.head()

,name,date,rating,subject,review
0,Masum Ahmed,31 January 2023,5.0 out of 5 stars,Awesome,
1,ImJ,8 December 2022,4.0 out of 5 stars,Its an Apple iPad,\nLoved the decent battery life and speakers a...
2,Mrinal,12 January 2023,5.0 out of 5 stars,Nothing comes close to this.,\nThere is literally no competition to iPad Ai...
3,Biresh Mohapatra,21 January 2023,5.0 out of 5 stars,Smooth running. Low screen quality,\nI don't like its screen quality and 🔋 life. ...
4,parmesh rawte,30 January 2023,5.0 out of 5 stars,best,\nbest ipad. for students\n


In [15]:
# Changing date column to pandas date format
new_df['date'] = pd.to_datetime(new_df['date'])

In [16]:
# we dont need name here so removing those columns
new_df.drop('name',inplace=True,axis=1)

In [17]:
# Now treating with rating column
new_df['rating'] = new_df['rating'].apply(lambda x: x[0])

In [18]:
new_df

,date,rating,subject,review
0,2023-01-31,5,Awesome,
1,2022-12-08,4,Its an Apple iPad,\nLoved the decent battery life and speakers a...
2,2023-01-12,5,Nothing comes close to this.,\nThere is literally no competition to iPad Ai...
3,2023-01-21,5,Smooth running. Low screen quality,\nI don't like its screen quality and 🔋 life. ...
4,2023-01-30,5,best,\nbest ipad. for students\n
...,...,...,...,...
144,2022-04-01,4,iPad with M1 is like next Gen,\nThis iPad is best in terms of performance an...
145,2022-08-05,5,#1 Computer Till Date,\nApple You are just fenomenally knew how to b...
146,2023-01-10,5,Nice product by apple,\nLikes : everything exceptDislike : can’t use...
147,2022-09-14,5,Apple no doubt product is good,\nApple quality never to be judged. It is good\n


In [19]:
# we can remove empty spaces with split method
new_df['subject']=new_df['subject'].apply(lambda x: (' '.join(x.split())).strip())
new_df['review']=new_df['review'].apply(lambda x: (' '.join(x.split())).strip())

In [20]:
# noticed '\n' is present at the first line of each row
new_df['review']=new_df['review'].apply(lambda x: (' '.join(x.split('\n'))).strip())


In [21]:
new_df.head()

,date,rating,subject,review
0,2023-01-31,5,Awesome,
1,2022-12-08,4,Its an Apple iPad,Loved the decent battery life and speakers and...
2,2023-01-12,5,Nothing comes close to this.,There is literally no competition to iPad Air ...
3,2023-01-21,5,Smooth running. Low screen quality,I don't like its screen quality and 🔋 life. Ot...
4,2023-01-30,5,best,best ipad. for students


In [22]:
# making all text columns to with all lower case
new_df['subject']=pd.DataFrame(new_df['subject'].apply(lambda x : x.lower()))
new_df['review']=pd.DataFrame(new_df['review'].apply(lambda x : x.lower()))

In [23]:
# Lets see most used words

words=[]
for i in new_df['review']:
    words.extend(i.split())

words,count=np.unique(words,return_counts=True)

most_used_words=sorted(zip(words,count), key=lambda item: item[1])[::-1]


In [24]:
limit=40
x=[]
y=[]
count=0
for i,j in most_used_words:
    if i[0].isalpha():
        count+=1
        x.append(i)
        y.append(j)
    if count==limit:
        break

In [25]:
fig=go.Figure(go.Bar(x=x,
                     y=y,
                     text=y,
                     marker=dict(color = y,
                                 colorscale='rainbow')))
fig.update_layout(showlegend=False,
                #   margin=dict(l=5, r=1, t=20, b=5),
                  xaxis=dict(
                            title='words',
                            showline=True,
                            showgrid=False,
                            showticklabels=True,
                            linecolor='#05050f'),
                  yaxis=dict(
                            title='counts',
                            showline=True,
                            showgrid=False,
                            showticklabels=True,
                            linecolor='#05050f'),
                  plot_bgcolor='white',
                  title='Most frequent words'
                 )
fig.show()

Seems like all of them are stop words

In [26]:
# Checking duplicate reviews
new_df[new_df['review'].duplicated()]

,date,rating,subject,review
11,2023-01-06,5,best in the budget,
21,2022-12-19,5,perfect,
37,2022-11-14,5,best in class tablet,
42,2022-11-01,5,very nice product. m1 made it an other level.,
53,2022-10-05,5,"original product, packed and shipped very care...",
66,2022-09-16,5,good,good
69,2022-08-30,5,good,
74,2022-08-07,5,best in class,
94,2022-05-15,5,it's good brightness and screen size making it...,good


In [27]:
# Seems like we have some null values in the data.
# Will trate this out in a moment
new_df.shape

(149, 4)

In [28]:
# so removing duplicate rows
new_df.drop_duplicates(subset=['review'],inplace=True)

In [29]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 0 to 148
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     140 non-null    datetime64[ns]
 1   rating   140 non-null    object        
 2   subject  140 non-null    object        
 3   review   140 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 5.5+ KB


In [30]:
# Cecking if review contains no information
new_df[new_df['review']=='']

,date,rating,subject,review
0,2023-01-31,5,awesome,


In [31]:

# droping those empty rows 
new_df.drop(new_df[new_df['review']==''].index,axis=0,inplace=True)

In [32]:
# verifying the result
new_df[new_df['review']=='']

,date,rating,subject,review


In [33]:
# Now all unique first letters
print(set([x[0] for x in new_df['review']]))

{'a', 'v', 'l', '-', 'd', 'g', 'f', 'w', 's', 'c', 'i', 'n', 'p', 'b', 'e', 'o', '2', 'r', 'm', 'आ', 't', 'u'}


In [34]:
# Some hindi words are there too
# We can find them

In [35]:
def is_hindi(character):
    maxchar = max(character)
    if u'\u0900' <= maxchar <= u'\u097f':
        return True
    else:
      return False

In [36]:
'\u097f'

'ॿ'

In [37]:
max('.*/@!aआaaaaaz>?]=+')

'आ'

In [38]:
is_hindi('आ'),is_hindi('az')

(True, False)

In [39]:
# There ew have sum of hindi reviews
x= new_df['review'].apply(lambda x : True if any(map(is_hindi,x.split())) else False)
np.sum(x)

1

Seems like only one review has written in Hindi


In [40]:
new_df.loc[x,'review'].values[0]

'आम आदमी के लिए यह प्रोडक्ट नहीं है अपना पैसा बर्बाद मत करें ज्यादा यूट्यूब की वीडियो देखकर अपना दिमाग खराब मत करें और मस्त एंड्रॉयड चलाओ सिंपल टेबलेट लो अपने काम को देखकर ही जरूरत के हिसाब से ही एप्पल की तरफ चलना चाहिए वरना आप निराश हो जाएंगे। पैसा भी बचाओ और उलझन से भी बचो।धन्यवाद।'

In [41]:
GoogleTranslator().translate(new_df.loc[x,'review'].values[0])

"This product is not for common man don't waste your money don't spoil your brain by watching more youtube videos and run cool android take simple tablet only after seeing your work you should go towards apple according to need otherwise you will be disappointed . Save money and also avoid confusion. Thank you."

In [42]:
# Now replacing hindi reviews
new_df.loc[x,'review']=GoogleTranslator().translate(new_df.loc[x,'review'].values[0])

In [43]:
# rechecking if any more hindi letter
np.sum(new_df['review'].apply(lambda x : True if any(map(is_hindi,x.split())) else False))

0

In [44]:
# Checking word count wrt date

fig=go.Figure(go.Scatter(x=new_df['date'], y=new_df['review'].apply(lambda x: len(x.split())),mode='markers')) 
fig.update_layout(title_text='Number of reviews per day')
fig.show()

In [45]:
# Checking ratings wrt date

fig=go.Figure(go.Scatter(x=new_df['date'], y=new_df['rating'], mode='markers')) 
fig.update_layout(title_text='ratings per day')
fig.show()

In [46]:
# Analyzing the words
nlp=spacy.load('en_core_web_sm')

In [47]:
# make reviews to single txt/string
review_txt = ''
for i in new_df['review']:
    review_txt = (review_txt + i ).strip()+ ' '
    

In [48]:
review_txt=nlp(review_txt)


In [49]:
# A dictionary that store all the POS( Parts of speech)
a_count_dict=dict()

for i in review_txt:
    if i.pos_ in a_count_dict.keys():
        a_count_dict[i.pos_] += 1
    else:
        a_count_dict[i.pos_] = 1

In [50]:
# These are all parts of speech we have
a_count_dict

{'VERB': 679,
 'DET': 445,
 'ADJ': 586,
 'NOUN': 1395,
 'CCONJ': 230,
 'ADV': 375,
 'PUNCT': 556,
 'SCONJ': 110,
 'PROPN': 192,
 'AUX': 451,
 'PRON': 522,
 'ADP': 535,
 'PART': 160,
 'NUM': 101,
 'INTJ': 12,
 'X': 6,
 'SYM': 5}

In [51]:
x=list(a_count_dict.values())
y=list(a_count_dict.keys())
fig=go.Figure(go.Bar(x=x,
                     y=y,
                     text=x,
                     orientation='h',
                     marker=dict(color=x,
                                    colorscale='dense') ))
fig.update_layout(title='POS count')
fig.show()

In [52]:
# defining a function to clean our string

def clean_it (txt):
  emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

  txt = re.sub('rt', '', txt) # remove 'RT' from reviews
  txt = re.sub('#[a-z0-9]+', '', txt) # remove the '#' from the reviews
  txt = re.sub('\\n', '', txt) # remove the '\n' character
  txt = re.sub('https?:\/\/\S+', '', txt) # remove the hyperlinks
  txt = re.sub('@[\S]*', '', txt) # remove @mentions
  txt = re.sub('^[\s]+|[\s]+$', '', txt) # remove leading and trailing whitespaces
  txt = re.sub(emoj, '', txt) # remove emojis
  txt = re.sub('[^a-z ]','',txt) #removes everything except alphabets
  return [x.strip() for x in txt.split()]

In [53]:
new_df['cleaned reviews']=new_df['review'].apply(clean_it)
new_df.head()

,date,rating,subject,review,cleaned reviews
1,2022-12-08,4,its an apple ipad,loved the decent battery life and speakers and...,"[loved, the, decent, battery, life, and, speak..."
2,2023-01-12,5,nothing comes close to this.,there is literally no competition to ipad air ...,"[there, is, literally, no, competition, to, ip..."
3,2023-01-21,5,smooth running. low screen quality,i don't like its screen quality and 🔋 life. ot...,"[i, dont, like, its, screen, quality, and, lif..."
4,2023-01-30,5,best,best ipad. for students,"[best, ipad, for, students]"
5,2022-12-31,5,what a wonderful piece of technology!!,"ipad is a very useful tool for reading, browsi...","[ipad, is, a, very, useful, tool, for, reading..."


In [54]:
# single word are not needed
new_df['cleaned reviews']=new_df['cleaned reviews'].apply(lambda x: [x for x in x if (len(x)>1)])

In [55]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kamal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
# Now removing stop words
new_df['cleaned reviews']=new_df['cleaned reviews'].apply(lambda y: [y for y in y if y not in set(stopwords.words('english'))] )

In [57]:
# Now removing empty reviews if any
new_df=new_df[new_df['cleaned reviews']!='']

In [58]:
# Now using lemmatizer
new_df['cleaned reviews']=new_df['cleaned reviews'].apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(y).strip() for y in x]) )
new_df.head()

,date,rating,subject,review,cleaned reviews
1,2022-12-08,4,its an apple ipad,loved the decent battery life and speakers and...,loved decent battery life speaker powerful pro...
2,2023-01-12,5,nothing comes close to this.,there is literally no competition to ipad air ...,literally competition ipad air tablet already ...
3,2023-01-21,5,smooth running. low screen quality,i don't like its screen quality and 🔋 life. ot...,dont like screen quality life otherwise smooth...
4,2023-01-30,5,best,best ipad. for students,best ipad student
5,2022-12-31,5,what a wonderful piece of technology!!,"ipad is a very useful tool for reading, browsi...",ipad useful tool reading browsing gaming etcbu...


In [59]:
# Sentiment Analysis
SIA = SentimentIntensityAnalyzer()

In [60]:
new_df['compound'] = new_df['cleaned reviews'].apply(lambda x: SIA.polarity_scores(x)['compound'])

In [61]:
new_df.head()

,date,rating,subject,review,cleaned reviews,compound
1,2022-12-08,4,its an apple ipad,loved the decent battery life and speakers and...,loved decent battery life speaker powerful pro...,0.4576
2,2023-01-12,5,nothing comes close to this.,there is literally no competition to ipad air ...,literally competition ipad air tablet already ...,0.6808
3,2023-01-21,5,smooth running. low screen quality,i don't like its screen quality and 🔋 life. ot...,dont like screen quality life otherwise smooth...,-0.1139
4,2023-01-30,5,best,best ipad. for students,best ipad student,0.6369
5,2022-12-31,5,what a wonderful piece of technology!!,"ipad is a very useful tool for reading, browsi...",ipad useful tool reading browsing gaming etcbu...,0.9682


In [62]:
# So here we can specify those tweets as positive where
# compound value is greater than 0
# if it is 0 then we can say it is neutral
# else negetive

new_df['Type']=new_df.compound.apply(lambda x: 'Negative' if x < 0 else 'Neutral' if x == 0 else 'Positive')

In [63]:
new_df.head()

,date,rating,subject,review,cleaned reviews,compound,Type
1,2022-12-08,4,its an apple ipad,loved the decent battery life and speakers and...,loved decent battery life speaker powerful pro...,0.4576,Positive
2,2023-01-12,5,nothing comes close to this.,there is literally no competition to ipad air ...,literally competition ipad air tablet already ...,0.6808,Positive
3,2023-01-21,5,smooth running. low screen quality,i don't like its screen quality and 🔋 life. ot...,dont like screen quality life otherwise smooth...,-0.1139,Negative
4,2023-01-30,5,best,best ipad. for students,best ipad student,0.6369,Positive
5,2022-12-31,5,what a wonderful piece of technology!!,"ipad is a very useful tool for reading, browsi...",ipad useful tool reading browsing gaming etcbu...,0.9682,Positive


In [64]:
# Checking the distribution of types
temp_df=new_df.Type.value_counts()
fig=go.Figure(go.Pie(labels=temp_df.index, values=temp_df.values))
fig.update_layout(title_text="Distribution of Types",
                  autosize=False)
fig.show()

In [65]:
# Compound value accross the dataframe
fig=go.Figure(go.Scatter(x=new_df.index, y=new_df.compound))
fig.update_layout(
                  xaxis=dict(
                            title='index'),
                  yaxis=dict(
                            title='compound'),
                  title='Compound value'
                 )
fig.show()

In [66]:
# Compound value wrt date
fig=go.Figure(go.Scatter(x=new_df.date, y=new_df.compound,mode='markers'))
fig.update_layout(
                  xaxis=dict(
                            title='date'),
                  yaxis=dict(
                            title='compound'),
                  title='Compound value every day'
                 )
fig.show()

In [67]:
# Lets see word_count vs compound

fig=go.Figure(go.Scatter(x=new_df['review'].apply(lambda x: len(x.split())), y=new_df.compound,mode='markers'))
fig.update_layout(xaxis=dict(
                            title='word count'),
                  yaxis=dict(
                            title='compound'),
                  title='word count vs compound',
                  autosize=False
                 )
fig.show()

In [68]:
# Now Vectorize our words/tweets

# Using CountVectorizer
CV=CountVectorizer()


In [69]:
# Splitting our data so as to ready for model building

y=new_df['Type'].apply(lambda x:{'Neutral':2,'Positive':3,'Negative':1}[x])
x_train,x_test,y_train,y_test=train_test_split((CV.fit_transform(new_df['cleaned reviews'])).toarray(),y,stratify=y,random_state=1)

In [70]:
classifiers = {
    "MultinomialNB": MultinomialNB(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(),
    "MLPClassifier": MLPClassifier(max_iter=1000),
    "AdaBoostClassifier": AdaBoostClassifier(),
}

In [71]:
for Algo, sklearn_classifier in classifiers.items():
    classifier = sklearn_classifier
    classifier.fit(x_train,y_train)
    
    print(f"{Algo} : Train- {accuracy_score(y_train,classifier.predict(x_train)):.2f}, Test- {accuracy_score(y_test,classifier.predict(x_test)):.2f}")

MultinomialNB : Train- 0.95, Test- 0.80
KNeighborsClassifier : Train- 0.74, Test- 0.71
DecisionTreeClassifier : Train- 1.00, Test- 0.74
RandomForestClassifier : Train- 1.00, Test- 0.71
LogisticRegression : Train- 0.98, Test- 0.74
MLPClassifier : Train- 1.00, Test- 0.71
AdaBoostClassifier : Train- 0.91, Test- 0.69


Choosing Logistic Regression as it is faster, simpler and scores well in sentiment analysis.

In [72]:
c_values=np.arange(0.02,0.0248,0.0001)
# c_values=np.arange(0.03,0.035,0.00001)


train_scores=[]
test_scores=[]

for i in c_values:
    test_model=LogisticRegression(C=i,multi_class='multinomial',random_state=11,n_jobs=-1)
    test_model.fit(x_train,y_train)
    
    train_scores.append(accuracy_score(y_train,test_model.predict(x_train)))
    test_scores.append(accuracy_score(y_test,test_model.predict(x_test)))

In [73]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=c_values,y=train_scores,name='train'))
fig.add_trace(go.Scatter(x=c_values,y=test_scores,name='test'))
fig.update_layout(hovermode='x unified')
fig.show()

In [74]:
# Choosing c= 0.0244
model=LogisticRegression(C=0.0244,random_state=11,multi_class='multinomial')
model.fit(x_train,y_train)

LogisticRegression(C=0.0244, multi_class='multinomial', random_state=11)

In [75]:

print(classification_report(y_train,model.predict(x_train)))


              precision    recall  f1-score   support

           1       1.00      0.05      0.10        19
           2       0.00      0.00      0.00        10
           3       0.73      1.00      0.84        75

    accuracy                           0.73       104
   macro avg       0.58      0.35      0.31       104
weighted avg       0.71      0.73      0.63       104

